In [4]:
import os
import torch
import cv2
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import preprocessing as prep
import validation
import plotting
import kornia as K
import kornia.feature as KF

input_dir = '../../data/train/' # directory of the training data

In [ ]:
# for use with AMD GPUs
# pip3 install torch --extra-index-url https://download.pytorch.org/whl/rocm5.1.1

In [5]:
# LoFTR config
default_cfg = {
    'backbone_type': 'ResNetFPN',
    'resolution': (8, 2),
    'fine_window_size': 5,
    'fine_concat_coarse_feat': True,
    'resnetfpn': {'initial_dim': 128, 'block_dims': [128, 196, 256]},
    'coarse': {
        'd_model': 256,
        'd_ffn': 256,
        'nhead': 8,
        'layer_names': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'],
        'attention': 'linear',
        'temp_bug_fix': False,
    },
    'match_coarse': {
        'thr': 0.2,
        'border_rm': 2,
        'match_type': 'dual_softmax',
        'dsmax_temperature': 0.1,
        'skh_iters': 3,
        'skh_init_bin_score': 1.0,
        'skh_prefilter': True,
        'train_coarse_percent': 0.4,
        'train_pad_num_gt_min': 200,
    },
    'fine': {'d_model': 128, 'd_ffn': 128, 'nhead': 8, 'layer_names': ['self', 'cross'], 'attention': 'linear'},
}

In [6]:
all_scenes = prep.get_scenes(input_dir)
all_scenes 

['brandenburg_gate',
 'british_museum',
 'buckingham_palace',
 'colosseum_exterior',
 'grand_place_brussels',
 'lincoln_memorial_statue',
 'notre_dame_front_facade',
 'pantheon_exterior',
 'piazza_san_marco',
 'sacre_coeur',
 'sagrada_familia',
 'st_pauls_cathedral',
 'st_peters_square',
 'taj_mahal',
 'temple_nara_japan',
 'trevi_fountain']

In [7]:
scenes = [all_scenes[0]]

In [15]:
pairs = prep.load_pairs(scenes,input_dir)
pairs = pairs.query('covisibility > 0.8')

loading category 1 of 1: brandenburg_gate


In [16]:
pairs

,pair,covisibility,fundamental_matrix,scene
0,90920828_5082887495-20133057_3035445116,0.905,-9.27158516e-04 1.22034601e-01 -9.87725452e+01...,brandenburg_gate
1,90920828_5082887495-17262282_1141017004,0.936,1.68899800e-03 1.18269247e-01 -1.02939518e+02 ...,brandenburg_gate
2,20133057_3035445116-17262282_1141017004,0.908,-5.56308440e-04 1.74170978e-02 -2.49391042e+01...,brandenburg_gate
3,38600512_2168650655-17262282_1141017004,0.901,1.26837683e-02 -5.42924979e-01 4.29820975e+02 ...,brandenburg_gate
4,60770994_853214983-17262282_1141017004,0.912,-1.05214974e-02 -4.87393290e-01 3.90468870e+02...,brandenburg_gate
...,...,...,...,...
127,60824110_1469539304-08468928_1469537330,0.811,8.91990074e-03 -2.94788540e-01 1.50446598e+02 ...,brandenburg_gate
129,66720928_3418295149-20133057_3035445116,0.809,-1.22423910e-03 5.86196722e-01 -4.42487049e+02...,brandenburg_gate
130,66720928_3418295149-02936509_94852264,0.811,3.41079145e-02 3.74560749e+00 -2.52775986e+03 ...,brandenburg_gate
131,66720928_3418295149-08814095_3507579332,0.802,1.56312064e-01 4.13537791e-01 -6.69635804e+02 ...,brandenburg_gate


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [11]:
# Initialize LoFTR
matcher = KF.LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.to(device).eval()

In [14]:
fund_matrix_list = []
pair_id_list = []

for index, row in pairs.head(10).iterrows():
    
    split_pair = pairs.pair[index].split('-')
    img_id0 = split_pair[0]
    img_id1 = split_pair[1]
    
    img0_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id0 + '.jpg'))
    img1_pth = os.path.join(input_dir, pairs.scene[index], "images", str(img_id1 + '.jpg'))
    img0 = prep.load_torch_image(img0_pth, device)
    img1 = prep.load_torch_image(img1_pth, device)
    batch = {"image0": K.color.rgb_to_grayscale(img0), 
            "image1": K.color.rgb_to_grayscale(img1)}
    
    with torch.no_grad():
        matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
        
    F = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.2, 0.99999, 50000)
    
    fund_matrix_list.append(" ".join(str(num) for num in F[0].flatten().tolist()))
    pair_id_list.append(";".join(["phototourism",pairs.scene[index],pairs.pair[index]]))
    
maa = validation.evaluate(input_dir, pair_id_list, fund_matrix_list)
print(f'mAA={maa:.05f} (n={len(pair_id_list)})')

/home/dieter/Documents/Python/capstone-project-image-matching-challenge/.venv/lib/python3.9/site-packages/kornia/feature/loftr/utils/coarse_matching.py:243: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  mkpts0_c = torch.stack([i_ids % data['hw0_c'][1], i_ids // data['hw0_c'][1]], dim=1) * scale0
/home/dieter/Documents/Python/capstone-project-image-matching-challenge/.venv/lib/python3.9/site-packages/kornia/feature/loftr/utils/coarse_matching.py:244: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for neg

mAA=0.20000 (n=10)


ToDo: 
- decompose F and plot
- save mkpts and mconf, so it can be used for drawing etc. later
- DBSCAN

## LoFTR example on single image pair

In [ ]:
# Initialize LoFTR and load two images
# matcher.eval() will notify all your layers that you are in eval mode, that way, batchnorm or dropout layers will work in eval mode instead of training mode.
matcher = KF.LoFTR(config=default_cfg)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.to("cpu").eval()

img0_pth = "../../data/test_images/1cf87530/0143f47ee9e54243a1b8454f3e91621a.png"
img1_pth = "../../data/test_images/1cf87530/a5a9975574c94ff9a285f58c39b53d2c.png"
img0_raw = cv2.imread(img0_pth, cv2.IMREAD_GRAYSCALE)
img1_raw = cv2.imread(img1_pth, cv2.IMREAD_GRAYSCALE)
img0_raw = cv2.resize(img0_raw, (640, 480))
img1_raw = cv2.resize(img1_raw, (640, 480))

img0 = torch.from_numpy(img0_raw)[None][None] / 255.
img1 = torch.from_numpy(img1_raw)[None][None] / 255.
batch = {'image0': img0, 'image1': img1}

In [ ]:
# Inference with LoFTR and get prediction
# torch.no_grad() impacts the autograd engine and deactivate it. It will reduce memory usage and speed up computations but you won’t be able to backprop (which you don’t want in an eval script).
with torch.no_grad():
    matcher(batch)
    mkpts0 = batch['mkpts0_f'].cpu().numpy()
    mkpts1 = batch['mkpts1_f'].cpu().numpy()
    mconf = batch['mconf'].cpu().numpy()

In [ ]:
# Draw
color = cm.jet(mconf)
text = [
    'LoFTR',
    'Matches: {}'.format(len(mkpts0)),
]
fig = plotting.make_matching_figure(img0_raw, img1_raw, mkpts0, mkpts1, color, text=text)